In [1]:
import numpy as np
from random import shuffle
train_input = ['{0:020b}'.format(i) for i in range(2**20)]

In [2]:
shuffle(train_input)
train_input = [map(int,i) for i in train_input]

In [3]:
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

In [4]:
train_output = []
 
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*21)
    temp_list[count]=1
    train_output.append(temp_list)

In [5]:
NUM_EXAMPLES = 10000
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:] #everything beyond 10,000
 
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES] #till 10,000

In [6]:
print len(train_input[0])
print len(train_output[0])

20
21


In [7]:
import tensorflow as tf

In [8]:
data = tf.placeholder(tf.float32, [None, 20,1])
target = tf.placeholder(tf.float32, [None, 21])

* Basic LSTM from https://arxiv.org/abs/1409.2329

<img align="left" width="600px" src='./basic_lstm.png' />

In [9]:
num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden,state_is_tuple=True)

val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

val = tf.transpose(val, [1, 0, 2])

last = tf.gather(val, int(val.get_shape()[0]) - 1)

weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))


# Softmax

 <img width='200px' align="left" src="./softmax.png" />

In [10]:
prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

# 什麼是 Corss Entropy 
- http://blog.csdn.net/rtygbwwwerr/article/details/50778098

In [11]:
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction,1e-10,1.0)))

# 建立 Optimize Function

In [12]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [13]:
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

In [14]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)


Instructions for updating:
Use `tf.global_variables_initializer` instead.


# 作 Training

In [15]:
batch_size = 1000
no_of_batches = int(len(train_input)/batch_size)
epoch =500
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize,{data: inp, target: out})
#     print "Epoch - ",str(i)
incorrect = sess.run(error,{data: test_input, target: test_output})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
# sess.close()


Epoch 500 error 0.4%


In [16]:
test_result = sess.run(prediction,{data: [inp[0]]})

In [17]:
count =0
for i in inp[0]:
    if i == [1]:
        count += 1
print count

7


In [18]:
test_result.argmax()

7